## K-means clustering and distance graph creation

In [1]:
import csv
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt 
%matplotlib inline

//anaconda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
//anaconda/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [4]:
def read_data(filename):
    with open(filename, 'r') as csvf:
        return [row for row in csv.reader(csvf)]

def get_cluster(title):
    return clusterIdOfSong[title]

In [5]:
X = read_data("songbysongtransprob.csv")
Y = []
for song in X:
    Y.append(song[0].replace('"','').replace("'",''))
    song.pop(0)

In [7]:
X_mat = np.array(X, dtype = np.float)
print(X_mat.shape)

(730, 144)


In [ ]:
"""6 Cluster K-Means Analysis"""
K = 6 #number of clusters
km = KMeans(n_clusters = K, n_init = 10000, random_state = 42)
km.fit(X_mat)  

In [ ]:
# Add center clusters to X, Y so that we can visualize them on the graph
Y = Y + ['CLUSTER0', 'CLUSTER1','CLUSTER2','CLUSTER3','CLUSTER4','CLUSTER5']
X =  np.concatenate((X_mat,km.cluster_centers_), axis = 0)

# Define dictionary of song names and song indices
song_dict = dict()
for i in range(0, len(Y)):
    song_dict[Y[i]] = X[i]
    
song_index = dict()
for i in range(0, len(Y)):
    song_index[Y[i]] = i
    
song_index['CLUSTER0'] = i+1
song_index['CLUSTER1'] = i+2
song_index['CLUSTER2'] = i+3
song_index['CLUSTER3'] = i+4
song_index['CLUSTER4'] = i+5
song_index['CLUSTER5'] = i+6

In [ ]:
# Compute distances between every two songs
import itertools
from scipy.spatial import distance

pairs_cosine = dict()
pairs_euclidean = dict()

counter = 0
for song_pair in itertools.combinations(Y, 2):
    pairs_cosine[song_pair] = distance.cosine(song_dict[song_pair[0]],song_dict[song_pair[1]])
    pairs_euclidean[song_pair] = distance.euclidean(song_dict[song_pair[0]],song_dict[song_pair[1]])
    counter += 1
print 'Number of pairs: ' + str(counter)

In [ ]:
#Normalize euclidean distances to 0-1 range
max_dist = max(pairs_euclidean.values())
min_dist = min(pairs_euclidean.values())

def min_max_scaler(x, min_, max_):
    return (x-min_)/(max_-min_)

In [ ]:
# Filter out NaN distances and scale to range 0-1 (only needed with Euclidean distance)
import math
graph_cosine = pairs_cosine.items()
graph_cosine = [i for i in graph_cosine if math.isnan(i[1]) == False]
graph_euclidean = pairs_euclidean.items()
graph_euclidean = [(i[0], min_max_scaler(i[1], min_dist, max_dist)) for i in graph_euclidean if math.isnan(i[1]) == False]

In [ ]:
# Plot distribution of distances
plt.figure(figsize=(10,5))
plt.ylabel('Number of distance pairs')
plt.xlabel('Distance')
plt.title('Histogram of cosine distance pairs')
plt.hist([i[1] for i in graph_cosine], bins = 50, color = 'green')
plt.show()

plt.figure(figsize=(10,5))
plt.ylabel('Number of distance pairs')
plt.xlabel('Distance')
plt.title('Histogram of euclidean distance pairs')
plt.hist([i[1] for i in graph_euclidean], bins = 50, color = 'green')
plt.show()

In [ ]:
# Manually create JSON file
json_nodes = '''"nodes": [''' + '\n'
for title, cluster in zip(Y, km.labels_):
    json_nodes += '''{"id": "''' + title + '''", "group": ''' + str(cluster) + '''},''' + '\n'
    
json_nodes = json_nodes[:-2] + '\n'+ ''']'''

In [ ]:
# Links using cosine distance
json_links = '''"links": [''' + '\n'
for song_pair, dist in graph_cosine:
    if dist < 0.9:
        json_links += '\t' + '''{"source": ''' + str(song_index[song_pair[0]]) + ''', "target": ''' + str(song_index[song_pair[1]]) + ''', "value": ''' + str(dist) + '''},''' + '\n' 
json_links = json_links[:-2] + '\n'+ ''']'''

json_string = '''{''' + json_nodes + ''',''' + json_links + '''}'''

text_file = open("graph_cosine.json", "w")
text_file.write(json_string)
text_file.close()

In [ ]:
# Links using euclidean distance
json_links = '''"links": [''' + '\n'
for song_pair, dist in graph_euclidean:
    if dist < 0.9:
        json_links += '\t' + '''{"source": ''' + str(song_index[song_pair[0]]) + ''', "target": ''' + str(song_index[song_pair[1]]) + ''', "value": ''' + str(dist) + '''},''' + '\n' 
json_links = json_links[:-2] + '\n'+ ''']'''

json_string = '''{''' + json_nodes + ''',''' + json_links + '''}'''

text_file = open("graph_euclidean.json", "w")
text_file.write(json_string)
text_file.close()